# Experimentations


In [ ]:
#!pip install visualkeras

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.datasets import mnist
from tensorflow.keras import layers

from involution import Involution
from utils import train_model, display_model, display_results

import visualkeras # for NN visualisation

## Experimentations on MNIST

### MNIST Dataset loading

In [ ]:
(x_raw_train_mnist, y_train_mnist), (x_raw_test_mnist, y_test_mnist) = mnist.load_data() # downloads the MNIST dataset (handwritten numbers)

# Scales images to the [0,1] range and expands the dimensions so that it has shape (28, 28, 1)
x_train_mnist = np.expand_dims(x_raw_train_mnist.astype("float32") / 255, -1) 
x_test_mnist = np.expand_dims(x_raw_test_mnist.astype("float32") / 255, -1)

In [ ]:
fig = plt.figure
id_img = np.random.randint(0, len(x_train_mnist)) # takes a random image from the dataset

plt.imshow(x_raw_train_mnist[id_img], cmap='gray') # imshow an image from the dataset
plt.suptitle("Representation of a "+str(y_train_mnist[id_img])+" in MNIST Dataset")
plt.show()

### Convolution

CNN inspired by Lecun et al in http://yann.lecun.com/exdb/publis/pdf/lecun-iscas-10.pdf.

Model available at https://keras.io/examples/vision/mnist_convnet/

In [ ]:
num_classes = 10

cnn_mnist = keras.Sequential( # definition of the CNN
    [
        keras.Input(shape=(28, 28, 1)), # images of size 28x28 pixels, with only 1 channel (greyscale)
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), # convolution layer: dot products with the weights, and activation function
        layers.MaxPooling2D(pool_size=(2, 2)), # pooling layer: downsampling, reduces the size of the representation
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(), # flatten layer: converts the data into a 1-D feature vector to feed it to the final layers
        layers.Dropout(0.5), # dropout layer: drops part of the data to avoid overfitting
        layers.Dense(num_classes, activation="softmax"), # dense layer: computes the result
    ]
)

cnn_mnist.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn_mnist.summary()

In [ ]:
display_model(cnn_mnist)

In [ ]:
batch_size = 128
epochs = 10

cnn_mnist_history, cnn_mnist_time = train_model(cnn_mnist, x_train_mnist, x_test_mnist, y_train_mnist, y_test_mnist, batch_size, epochs)

In [ ]:
display_results(cnn_mnist_history, cnn_mnist_time, "CNN")

### Involution

In [ ]:
inn_mnist = keras.Sequential( # definition of the INN
    [
        keras.Input(shape=(28, 28, 1)), # images of size 28x28 pixels, with only 1 channel (greyscale)
        Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
        layers.MaxPooling2D(pool_size=(2, 2)), # pooling layer: downsampling, reduces the size of the representation
        Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(), # flatten layer: converts the data into a 1-D feature vector to feed it to the final layers
        layers.Dropout(0.5), # dropout layer: drops part of the data to avoid overfitting
        layers.Dense(num_classes, activation="softmax"), # dense layer: computes the result
    ]
)

inn_mnist.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
batch_size = 128
epochs = 10

inn_mnist_history, inn_mnist_time = train_model(inn_mnist, x_train_mnist, x_test_mnist, y_train_mnist, y_test_mnist, batch_size, epochs)

In [ ]:
inn_mnist.summary()
display_model(inn_mnist)

In [ ]:
display_results(inn_mnist_history, inn_mnist_time, "INN")

## Experimentations on CIFAR10

### CIFAR10 Dataset loading

In [ ]:
(x_train_cifar, y_train_cifar), (x_test_cifar, y_test_cifar) = tf.keras.datasets.cifar10.load_data()

class_img = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse' , 'ship', 'truck']
num_classes = len(class_img)

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train_cifar / 255.0, x_test_cifar / 255.0

# Creating datasets
batch_size = 256
train_ds = tf.data.Dataset.from_tensor_slices((x_train_cifar, y_train_cifar)).shuffle(batch_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices( (x_test_cifar, y_test_cifar)).batch(batch_size)

### Convolution

In [ ]:
cnn_cifar = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes),
    ]
)

cnn_cifar.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

cnn_cifar.summary()

In [ ]:
display_model(cnn_cifar)

In [ ]:
batch_size = 256
epochs = 10

tf.config.run_functions_eagerly(True) # needed to solve a keras bug
cnn_cifar_history, cnn_cifar_time = train_model(cnn_cifar, x_train_cifar, x_test_cifar, y_train_cifar, y_test_cifar, batch_size, epochs)

In [ ]:
display_results(cnn_cifar_history, cnn_cifar_time, "CNN")

### Involution

In [ ]:
inn_cifar = tf.keras.models.Sequential([
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu1"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu2"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu3"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes),
])

inn_cifar.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
batch_size = 256
epochs = 10

inn_cifar_history, inn_cifar_time = train_model(inn_cifar, x_train_cifar.astype("float"), x_test_cifar.astype("float"), y_train_cifar.astype("float"), y_test_cifar.astype("float"), batch_size, epochs)

In [ ]:
inn_cifar.summary()

In [ ]:
display_results(inn_cifar_history, inn_cifar_time, "INN")

## Experimentations on fashion MNIST

### Fashion MNIST data loading

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
# Import
((x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion)) = fashion_mnist.load_data()

# Normalization between 0 and 1
x_train_fashion, x_test_fashion = x_train_fashion / 255.0, x_test_fashion / 255.0

x_train_fashion = np.expand_dims(x_train_fashion.astype("float32"), -1) 
x_train_fashion = np.expand_dims(x_test_fashion.astype("float32"), -1)

class_names = ['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.imshow(x_train_fashion[20].reshape(28,28),cmap = 'gray' )
plt.xlabel(class_names[y_train_fashion[20]])
plt.show()

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

((x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion)) = fashion_mnist.load_data()
x_train_fashion, x_test_fashion = x_train_fashion / 255.0, x_test_fashion / 255.0
class_names = ['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### Convolution

In [ ]:
cnn_fashion = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (28,28,1), padding = 'same'),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(64, activation = 'relu'),
          tf.keras.layers.Dense(10),])

cnn_fashion.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

cnn_fashion.summary()

In [ ]:
display_model(cnn_fashion)

In [ ]:
batch_size = 128
epochs = 10

cnn_fashion_history, cnn_fashion_time = train_model(cnn_fashion, x_train_fashion, x_test_fashion, y_train_fashion, y_test_fashion, batch_size, epochs)

In [ ]:
display_results(cnn_fashion_history, cnn_fashion_time, "CNN")

### Involution

In [ ]:
inn_fashion = tf.keras.models.Sequential([
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu1"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu2"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu3"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10),
])

inn_fashion.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
x_train_fashion = np.expand_dims(x_train_fashion.astype("float32"), -1) 
x_test_fashion = np.expand_dims(x_test_fashion.astype("float32"), -1)

In [ ]:
batch_size = 128
epochs = 10

tf.config.run_functions_eagerly(True)
inn_fashion_history, inn_fashion_time = train_model(inn_fashion, x_train_fashion, x_test_fashion, y_train_fashion, y_test_fashion, batch_size, epochs)

In [ ]:
inn_fashion.summary()

In [ ]:
display_results(inn_fashion_history, inn_fashion_time, "INN")